# Pipeline test notebook

### Import requirements

In [29]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import time

from ImageProcessor import ImageProcessor
from StereoMatcher import StereoMatcher
from VoxelGrid import VoxelGrid

%matplotlib ipympl

### Load calibrations and other data

In [30]:
imageProcessor = ImageProcessor()
imageProcessor.loadMonoCalibrationResults()
imageProcessor.loadStereoCalibration()
imageProcessor.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [31]:
stereoMatcher = StereoMatcher(imageProcessor=imageProcessor, \
                matcher="SGBM", vertical=True, createRightMatcher=False)

imageProcessor.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Loading images

In [32]:
path = "captures/testImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-9


### Select image pair and display

In [33]:
imageNumber = 1

imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

plt.figure()
plt.imshow(cv2.cvtColor(np.hstack([imageL, imageL]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Convert to grayscale and undistort

In [34]:
imageProcessor.convertToGrayscale(imageL, imageR)
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

### View undistorted image

In [35]:
fig = plt.figure()
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(np.hstack([imageProcessor.undistortImageL, \
            imageProcessor.undistortImageR]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
fig = plt.figure()
fig.suptitle("horizontal epipolar")
plt.imshow(cv2.cvtColor(imageProcessor.drawHorEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
fig = plt.figure(figsize=(6,15))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(imageProcessor.drawVertEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute disparity map

In [38]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 5.0
maxDisparity: 37.0


### View disparity map

In [39]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE), cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [40]:
focalLength = imageProcessor.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

5.0
37.0
(640, 360)
533.37445
3946.9712


### View depth map

In [41]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE), cmap="Greys")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
# Use to save images
# _=cv2.imwrite("depth.png", cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

### Compute point cloud

In [43]:
voxelGrid = VoxelGrid(stereoMatcher, imageProcessor)
voxelGrid.verbose = True

In [44]:
voxelGrid.generatePointCloud()

Points in unfiltered pointcloud: 11520; completed in 0.02803 sec


Filtering extreme points

In [45]:
voxelGrid.filterPointCloud()

Points in filtered pointcloud: 8706; completed in 0.00552 sec


Rotate point cloud to have y forward

In [46]:
voxelGrid.redefinePointCloudCoordinate()

### View point cloud

In [47]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.pointCloud[:,0], voxelGrid.pointCloud[:,1], \
    voxelGrid.pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_xlim(-1000,1000)
ax.set_ylim(0,2000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [48]:
voxelGrid.resetVoxelGrid()
voxelGrid.voxelizePointCloud()

Voxels in grid: 93; completed in 0.11463 sec; 110 iterations


### View voxelized point cloud

In [49]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.voxelGrid[:,0], \
        voxelGrid.voxelGrid[:,1], \
        voxelGrid.voxelGrid[:,2])

ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_xlim(-1000,1000)
ax.set_ylim(0,2000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …